In [1]:
import glob
import os
from preprocess import normalize_text, remove_punctuations, tokenize_text, stopword_remover

In [2]:
def preprocess(text):
    text = remove_punctuations(text)
    text = normalize_text(text)
    tokens = tokenize_text(text)
    tokens = stopword_remover(tokens)
    return " ".join(tokens)

In [3]:
file_dir = r'data\**\*.txt'
file_paths = glob.glob(file_dir)
print(len(file_paths))

def readfile(filename):
  from functools import reduce
  with open(filename, 'r', encoding='utf-8') as f:
      lines = f.readlines()
      text = reduce(lambda a, b : a + " " +str(b), lines)
  return text

textlist = []
for file in file_paths:
  textlist.append(readfile(file))

30


In [4]:
from sklearn.model_selection import train_test_split

def assign_unique_ids(input_list):
    unique_ids = {}
    result = []
    current_id = 0
    
    for element in input_list:
        if element not in unique_ids:
            unique_ids[element] = current_id
            current_id += 1
        result.append(unique_ids[element])
    
    return result

X, y = [], []
for text in textlist:
    X.append(preprocess(text))

y = assign_unique_ids([file_path.split('\\')[1] for file_path in file_paths])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y, shuffle = True)
print(len(X_train), len(X_test), len(y_train), len(y_test))

24 6 24 6


In [5]:
X

['खजूर वृक्षों छोटी सी छाया कड़ाके धूप सिकुड़ आपमें पेड़ पैरों तले छिपी रही उत्तप्त साँस छटपटाते वातावरण चार केना फूलों आभा तरलता चिकनेपन भ्रम उत्पन्न रही यद्यपि सब सूनापन प्यासापन रुखाई केना फूलों पास छींट टुकड़े कंधे ढँके मेरिया बैठी उससे दूर भूमि अख़बार बिछाए उसकी छोटी बहन कार्मेन रूमाल काढ़ रही दोनों ध्यान मस्त किंतु ध्यान विषय विभिन्न दृष्टियों देख यद्यपि स्वयं बात जानतीं विचार कितने पास मँडरा यद्यपि मेरिया कभी स्वीकार करेगी क्योंकि हृदय गुप्ततम रहस्य समझती कार्मेन आँखें हाथ रूमाल लगी लाल धागे नाम काढ़ रही मेहँदी रंग लिखा मिगेल नाम चारों बेल काढ़ी चुकी बेल ऊपर लाल झंडा मेरिया पास चीज़ चर्मचक्षुओं देख रही केना फूलों आगे खजूर चार झुरमुट आगे छोटे छोटे नए गन्ने खेत पार कहीं स्पष्ट किंतु अदृश्य सत्यताएँ उसकी आँखें गड़ी मार काट हत्या भूख प्यास विद्रोह मेरिया देख रही स्वप्न छाया देख रही स्वप्न टूट चुका किंतु बिखरा बद्ध चुका किंतु मरा मिगेल याद रही मिगेल जेल बैठा मिगेल क्या मन उलझाने स्पष्ट विचार आवश्यक क्या कवि कविता लिखने लिखने विचार अनुकूल झुकाव इतना तल्लीन कविता अभिव्यक्ति अकिंचन आकस्

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# frequecy, select k best, pca, tsne

# Initialize CountVectorizer with n-gram range (e.g., 2-grams)
ngram_range = (2, 3)  # You can adjust the range as needed
vectorizer = CountVectorizer(ngram_range=ngram_range)

# Fit and transform the stories into n-gram features
X = vectorizer.fit_transform(X_train)

# Convert the result to a DataFrame for better visualization
df_ngram_features = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Display the n-gram features
print(df_ngram_features)

    10 15  10 15 नट  10 नट  10 नट चल  10 नट समय  10 बज  10 बज लड  124 कर  \
0       0         0      0         0          0      0         0       0   
1       0         0      0         0          0      0         0       0   
2       0         0      0         0          0      0         0       0   
3       0         0      0         0          0      0         0       0   
4       0         0      0         0          0      0         0       0   
5       0         0      0         0          0      0         0       0   
6       0         0      0         0          0      0         0       0   
7       0         0      2         1          1      0         0       0   
8       0         0      0         0          0      0         0       0   
9       0         0      0         0          0      0         0       1   
10      0         0      0         0          0      0         0       0   
11      0         0      0         0          0      0         0       0   
12      0   

In [9]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained mBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

c:\Users\dhruv\miniconda3\envs\temp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\dhruv\miniconda3\envs\temp\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
inputs = tokenizer(textlist[0], return_tensors='pt')

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)
    # Last hidden states
    last_hidden_states = outputs.last_hidden_state

# Extract token embeddings
token_embeddings = last_hidden_states.squeeze().cpu().numpy()

RuntimeError: The size of tensor a (21714) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
# Token IDs and tokens
token_ids = inputs['input_ids'].squeeze().tolist()
tokens = tokenizer.convert_ids_to_tokens(token_ids)

# Print tokens with their embeddings
for token, embedding in zip(tokens, token_embeddings):
    print(f"Token: {token}, Embedding: {embedding.shape}...")  # Print first 5 values for brevity

In [19]:
import numpy as np

def get_bert_embeddings_for_long_text(text, tokenizer, model, max_length=512):
    # Tokenize and encode the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=False, max_length=max_length)
    input_ids = inputs['input_ids'][0]  # Get the input IDs as a tensor
    attention_mask = inputs['attention_mask'][0]  # Get the attention mask

    # Calculate the number of chunks
    num_chunks = (len(input_ids) + max_length - 1) // max_length
    chunk_embeddings = []
    print(num_chunks)
    with torch.no_grad():
        for i in range(num_chunks):
            # Get the chunk of input IDs and attention mask
            chunk_input_ids = input_ids[i * max_length: (i + 1) * max_length].unsqueeze(0)
            chunk_attention_mask = attention_mask[i * max_length: (i + 1) * max_length].unsqueeze(0)

            # Get the output for this chunk
            outputs = model(input_ids=chunk_input_ids, attention_mask=chunk_attention_mask)
            
            # Get the embedding for the [CLS] token
            chunk_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
            chunk_embeddings.append(chunk_embedding)
    
    # Average the embeddings for all chunks
    avg_embedding = np.mean(chunk_embeddings, axis=0)
    
    return avg_embedding


In [26]:
embedding = get_bert_embeddings_for_long_text(X_train[0], tokenizer, model)
print(embedding.shape)

c:\Users\dhruv\miniconda3\envs\temp\lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


6
(768,)


In [25]:
len(.split())

1091